In [15]:
import os
import json
import psycopg2
from dotenv import load_dotenv
from datetime import datetime
from collections import defaultdict

# Cargar variables desde .env
load_dotenv()

DB_CONFIG = {
    "host": os.getenv("DB_HOST", "localhost"),
    "port": int(os.getenv("DB_PORT", 5432)),
    "dbname": os.getenv("DB_NAME", "postgres"),
    "user": os.getenv("DB_USER", "postgres"),
    "password": os.getenv("DB_PASSWORD", "postgres123"),
}

# Carpeta con los JSON
JSON_FOLDER = "./json_files"

def parse_filename(filename):
    """
    Ejemplo: Riohacha_11_538415_-72_916784_1722470400_1754092799_xxx.json
    """
    parts = filename.split("_")
    city = parts[0]
    lat = float(parts[1] + "." + parts[2])
    lon = float(parts[3] + "." + parts[4])
    start = int(parts[5])
    end = int(parts[6])
    return city, lat, lon, start, end

def process_daily_format(daily, location_id, cur):
    """Procesa un JSON formato daily (arrays por día)"""
    for date, tmax, tmin, prec in zip(
        daily["time"],
        daily["temperature_2m_max"],
        daily["temperature_2m_min"],
        daily["precipitation_sum"]
    ):
        cur.execute("""
            INSERT INTO weather_daily (location_id, date, temp_max, temp_min, precipitation)
            VALUES (%s, %s, %s, %s, %s);
        """, (location_id, date, tmax, tmin, prec))

def process_hourly_format(data_list, location_id, cur):
    """Procesa un JSON formato horario (muchos registros con dt/main) agrupando por fecha"""
    daily_data = defaultdict(lambda: {"tmax": float('-inf'),
                                      "tmin": float('inf'),
                                      "prec": 0})
    for record in data_list:
        date = datetime.utcfromtimestamp(record["dt"]).date()
        tmax = record["main"]["temp_max"]
        tmin = record["main"]["temp_min"]
        prec = 0
        if "rain" in record:
            prec = record["rain"].get("1h", 0)
        daily_data[date]["tmax"] = max(daily_data[date]["tmax"], tmax)
        daily_data[date]["tmin"] = min(daily_data[date]["tmin"], tmin)
        daily_data[date]["prec"] += prec

    for date, vals in daily_data.items():
        cur.execute("""
            INSERT INTO weather_daily (location_id, date, temp_max, temp_min, precipitation)
            VALUES (%s, %s, %s, %s, %s);
        """, (location_id, date, vals["tmax"], vals["tmin"], vals["prec"]))

def load_json_to_db(file_path, conn):
    filename = os.path.basename(file_path)
    city, lat, lon, start, end = parse_filename(filename)

    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    with conn.cursor() as cur:
        # Insertar metadata de ubicación
        cur.execute("""
            INSERT INTO weather_location (city, lat, lon, start, "end")
            VALUES (%s, %s, %s, %s, %s)
            RETURNING id;
        """, (city, lat, lon, start, end))
        location_id = cur.fetchone()[0]

        # Detectar formato
        if isinstance(data, dict) and ("daily" in data or "data" in data):
            daily = data.get("daily") or data.get("data")
            process_daily_format(daily, location_id, cur)

        elif isinstance(data, list):
            # puede ser lista de registros horarios o daily anidado
            primer = data[0]
            if "daily" in primer or "data" in primer:
                daily = primer.get("daily") or primer.get("data")
                process_daily_format(daily, location_id, cur)
            elif "main" in primer and "dt" in primer:
                process_hourly_format(data, location_id, cur)
            else:
                raise ValueError(f"Formato no reconocido en {filename}")
        else:
            raise ValueError(f"Formato no reconocido en {filename}")

    conn.commit()
    print(f"✅ {filename} cargado en la BD (location_id={location_id})")

def main():
    conn = psycopg2.connect(**DB_CONFIG)
    try:
        for filename in os.listdir(JSON_FOLDER):
            if filename.endswith(".json"):
                file_path = os.path.join(JSON_FOLDER, filename)
                load_json_to_db(file_path, conn)
    finally:
        conn.close()

if __name__ == "__main__":
    main()


C:\Users\valentin.e.bussolini\AppData\Local\Temp\ipykernel_23556\3316067583.py:53: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(record["dt"]).date()


✅ Patagonia_-41_810147_-68_906269_1722470400_1754092799_689025ec11032e000839b7c9.json cargado en la BD (location_id=4)
✅ Riohacha_11_538415_-72_916784_1722470400_1754092799_689025ec11032e000839b7c9.json cargado en la BD (location_id=5)
